## Network Biology 
### Coding Assignment 3
#### Submitted By: Divyanshu Srivastava

In [2]:
import networkx as nx
import numpy as np
import matplotlib as plt
import random
import copy
from Bio.PDB import *
from IPython.display import HTML, display
import tabulate

#### Question 1


In [3]:
## Initialization

genes = 25
tfbs_pool = 10
tfbs_per_gene = 3

gene_pool = {}
for g in range(genes):
    gene_pool[g] = random.sample(range(tfbs_pool), tfbs_per_gene)

steps_of_evolution = 100

p_gene_dublication = 0.25
p_gene_deletion = 0.25
p_tfbs_dublication = 0.25
p_tfbs_deletion = 0.25
p_vector = [p_gene_dublication, p_gene_deletion, p_tfbs_dublication, p_tfbs_deletion]



## Evolution
for s in range(steps_of_evolution):
    r = np.random.choice(len(p_vector), p = p_vector)
#     print gene_pool
    if r == 0:
#         print "Evolution Step : " + str (s) + " Gene Dublication"
        gene_to_dublicate = random.sample(range(genes), 1)[0]
        gene_pool[genes] = copy.deepcopy(gene_pool[gene_to_dublicate])
        genes = genes + 1;
    elif r == 1:
#         print "Evolution Step : " + str (s) + " Gene Deletion"
        gene_to_delete = random.sample(range(genes), 1)[0]
#         gene_pool.pop(gene_to_delete)
        for i in range(gene_to_delete, genes-1):
            gene_pool[i] = copy.deepcopy(gene_pool[i+1])
#         if not gene_to_delete == genes - 1:
        gene_pool.pop(genes - 1)
        genes = genes-1
        if genes == 0:
            print "Gene Pool Empty !"
            break
    elif r == 2:
#         print "Evolution Step : " + str (s) + " TFBS Dublication"
        tfbs_probability = np.array(range(0, tfbs_pool))
        for g in gene_pool:
            for value in gene_pool[g]:
                tfbs_probability[value] = tfbs_probability[value]+1
        tfbs_probability = tfbs_probability.astype(np.float)
        tfbs_probability = tfbs_probability / np.sum(tfbs_probability)
        tfbs_to_dublicate = np.random.choice(tfbs_pool, p = tfbs_probability)
        flag = False
        while not flag:
            gene_target = np.random.choice(gene_pool.keys())
            if tfbs_to_dublicate not in gene_pool[gene_target]:
                gene_pool[gene_target].append(tfbs_to_dublicate)
                flag = True
    else:
#         print "Evolution Step : " + str (s) + " TFBS Deletion"
        gene_target = np.random.choice(gene_pool.keys())
#         print "gene_target = " + str(gene_target)
        tfbs_to_delete = np.random.choice(gene_pool[gene_target])
#         print "tfbs_to_delete = " + str(tfbs_to_delete)
        gene_pool[gene_target].remove(tfbs_to_delete)
#         print "gene pool agter deletion"
#         print gene_pool
        if len(gene_pool[gene_target]) == 0:
            
            gene_to_delete = gene_target
#             print "removing gene " + str(gene_to_delete)
#             gene_pool.pop(gene_to_delete)
            for i in range(gene_to_delete, genes-1):
#                 print i
                gene_pool[i] = copy.deepcopy(gene_pool[i+1])
#             if not gene_to_delete == genes - 1:
            gene_pool.pop(genes - 1)
            genes = genes-1
            if genes == 0:
                print "Gene Pool Empty !"
                break


                
## Building coevolution network
G = nx.Graph()
for g_a in gene_pool.keys():
    for g_b in gene_pool.keys():
        if not g_a == g_b: 
            if len(set(gene_pool[g_a]).intersection(gene_pool[g_b])) > 0:
                G.add_edge(g_a, g_b)
gene_pool


{0: [4, 0, 3, 9],
 1: [4, 3, 0],
 2: [5, 4],
 3: [9, 4, 1, 3],
 4: [2, 3, 5, 6],
 5: [8, 7, 9, 0],
 6: [0, 4],
 7: [4, 7, 0, 5],
 8: [0],
 9: [5, 3, 2, 6, 8],
 10: [4, 0, 8],
 11: [2, 1, 5],
 12: [4, 0, 3],
 13: [0, 4, 5, 9, 3],
 14: [4, 0],
 15: [0, 3, 8],
 16: [4, 7, 6],
 17: [4, 3, 0],
 18: [2, 3, 5, 6],
 19: [3, 0],
 20: [0, 9],
 21: [4, 3, 0],
 22: [4, 1, 3],
 23: [2, 3, 5, 6]}

#### Question 2


In [4]:
def get_RIG(coordinates, labels, cut_off):
    """this function computes residue interaction graphs"""
    RIG = nx.Graph()
    label_ids = range(len(labels))
    RIG.add_nodes_from(label_ids)
    for i in label_ids:
        for j in label_ids:
            if not i == j:
                if np.linalg.norm(coordinates[i] - coordinates[j]) < 7:
                    RIG.add_edge(i, j)
    return RIG

def get_LIN(RIG, threshold):
    """this function computes long range network"""
    LIN = nx.Graph(RIG)
    for e in LIN.edges():
        if not abs(e[0] - e[1]) == 1:
            if abs(e[0] - e[1]) < threshold:
                LIN.remove_edge(e[0], e[1])
    return LIN

In [11]:
RIG_CUTOFF = 7
LIN_THRESHOLD = 12

parser = PDBParser()
pdb_files = ['1csp.pdb', '1hrc.pdb', '1pks.pdb', '2abd.pdb','3mef.pdb']
RIG = []
LIN = []


for pdb_file in pdb_files:
    structure = parser.get_structure('pdb_file', 'pdb files/' + pdb_file)
    coordinates = []
    labels = []
    for model in structure:
        for chain in model:
            for residue in chain:
                try:
                    coordinates.append(residue['CA'].get_coord())
                    labels.append(residue.get_resname())
                except KeyError:
                    pass
            RIG.append(get_RIG(coordinates, labels, RIG_CUTOFF))
            LIN.append(get_LIN(RIG[-1], LIN_THRESHOLD))
            break     ## working on chain id A only
        break         ## Working on model id 0 only
        
        
output = [['PBD ID', 'Nodes', 'Edges (RIG)', 'L (RIG)', 'C (RIG)', 'Edges (LIN)', 'L (LIN)', 'C (LIN)']]
for i in range(len(pdb_files)):
    append_list = [pdb_files[i], RIG[i].number_of_nodes(), RIG[i].number_of_edges()]
    append_list.append(nx.average_shortest_path_length(RIG[i]))
    append_list.append(nx.average_clustering(RIG[i]))
    append_list.append(LIN[i].number_of_edges())
    append_list.append(nx.average_shortest_path_length(LIN[i]))
    append_list.append(nx.average_clustering(LIN[i]))
    output.append(append_list)

display(HTML(tabulate.tabulate(output, tablefmt='html')))

PBD ID,Nodes,Edges (RIG),L (RIG),C (RIG),Edges (LIN),L (LIN),C (LIN)
1csp.pdb,67,255,3.0447761194,0.561993313486,146,4.61239258254,0.326143094054
1hrc.pdb,104,400,3.91486183719,0.566917484225,182,6.77445855116,0.227335164835
1pks.pdb,76,304,3.08070175439,0.557453657454,182,4.38877192982,0.332002354371
2abd.pdb,86,334,3.75923392613,0.57667093888,140,6.09083447332,0.125442967885
3mef.pdb,69,248,3.25532821824,0.574875527049,145,4.90707587383,0.289786059351


##### Observations
1csp and 1pks proteins show high cllustering coefficients in LIN models, suggesting that they have lower rate of foldings. 

#### Question 3


#### Question 4
